### +++ P03 static +++

In [1]:
import opensim as osim
import os
import sys
from simFunctions import *
from subprocess import call
from stan_utils import subject_specific_isometric_force
from stan_utils import *

# 2. Scaling

For walking, the model is scaled following distances between external markers and muscle forces are scaled by a factor calculated with mass and height of the subject relative the mass and the height of the generic model.

## 2.1 Scale model to height and weight

In [2]:
# subject data
subject_height = 1.6
generic_height = 1.77

In [3]:
cmdprog = 'opensim-cmd'
cmdtool = 'run-tool'
cmdfile = '..\p03_scale_settings_2.xml'
cmdfull = [cmdprog, cmdtool, cmdfile]
rc = runProgram(cmdfull)

c:\Users\Ekatarina\miniconda3\envs\py38\lib\subprocess.py:848: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)


[info] Preparing to run ScaleTool.

[info] Processing subject Mixed_Marker_Ratio_Scale...

[info] Step 1: Loading generic model

[info] Loaded model RajagopalModifiedGeneric from file c:\Users\Ekatarina\Dropbox\My_PC_LAPTOP-OF7GLRAD\Documents\WORK\hip\MotionStudy\MotionData\003\P03\RajagopalModified_generic_copy.osim

[info] Loading marker set from '../scaling_EXP_Markers_to_Scale_final_3.xml'.

[info] Updated markers in model RajagopalModifiedGeneric

[info] Step 2: Scaling generic model

[info] Loaded marker file ../Session2/Static03/marker_experimental.trc (40 markers, 660 frames)

[info] Measurement 'torso height'

[info] 	pair 0 (PE01, C7): model = 0.439607, experimental = 0.410544

[info] 	overall scale factor = 0.93389

[info] Measurement 'foot_horiz_l'

[info] 	pair 0 (LHEE, LTOE): model = 0.212119, experimental = 0.206409

[info] 	overall scale factor = 0.973083

[info] Measurement 'foot_horiz_r'

[info] 	pair 0 (RHEE, RTOE): model = 0.204784, experimental = 0.204249

[info] 	

## 2.2 Scale model muscle forces to height and weight for walking

In [6]:
generic_model = osim.Model('../RajagopalModified_generic_copy.osim')
model=osim.Model(os.path.join('../p03_scaled_2.osim'))
scaled = scaleOptimalForceSubjectSpecific(generic_model, model, 1.77, 1.60)

In [7]:
pelvis_coordinates = model.getCoordinateSet().get('pelvis_tilt')
pelvis_coordinates.setRangeMax(4)
scaled.printToXML(os.path.join('..\\p03_forces_scaled_2.osim'))

True

## 2.3 Increase model muscle forces for running

In [8]:
model=osim.Model(os.path.join('../p03_forces_scaled_2.osim'))
muscles=model.getMuscles()
muscles.getSize()

80

'glmax1_l', 'glmax1_r', 'glmax2_l', 'glmax2_r', 'glmax3_l', 'glmax3_r', 'glmed1_l', 'glmed1_r', 'glmed1_2', 'glmed2_r', 'glmed3_l', 'glmed3_r', 'glmin1_l', 'glmin1_r', 'glmin2_l', 'glmin2_r', 'glmin3_l', 'glmin3_r', 'semimem_l', 'semimem_r', 'semiten_l', 'semiten_r', 'soleus_l', 'soleus_r', 'tibpost_l', 'tibpost_r', 'vaslat_l', 'vaslat_r', 'vasmed_l', 'vasmed_r', 'vasint_l', 'vasint_r'

In [9]:
good_muscles = []

In [10]:
for i in range(0, muscles.getSize()):
    currentMuscle = muscles.get(i)
    if currentMuscle.getName() not in good_muscles:
        forceScaleFactor = 2
        currentMuscle.setMaxIsometricForce(forceScaleFactor * currentMuscle.getMaxIsometricForce() )        

In [11]:
pelvis_coordinates = model.getCoordinateSet().get('pelvis_tilt')
pelvis_coordinates.setRangeMax(4)
pelvis_coordinates.setRangeMin(-4)

In [12]:
model.printToXML(os.path.join('../p03_forces_scaled_2_running.osim'))

True